# Prediction <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

## Background

stuff

## Description
This notebook

### Load Packages

In [2]:
import datacube
from odc.algo import xr_geomedian
import xarray as xr
import subprocess as sp
import numpy as np
from joblib import load
import geopandas as gpd
from datacube.utils import geometry
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs

import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_classificationtools import predict_xr#,predict_proba_xr
from deafrica_dask import create_local_dask_cluster
from deafrica_plotting import map_shapefile
from deafrica_bandindices import calculate_indices
from deafrica_temporal_statistics import temporal_statistics

sys.path.append('../datacube-2nd-order-stats')
from model import TernaryMAD

### Set up a dask cluster
This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the dask dashboard, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.

In [3]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:33157 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


## Analysis parameters

* `ncpus`: Set this value to > 1 to parallize the collection of training data. eg. npus=8. 
* `model`: Set

In [4]:
# automatically detect number of cpus, adjust to [-3:] if working on deafault Sandbox
ncpus= int(float(sp.getoutput('env | grep CPU')[-4:]))

model_path = 'results/ml_model.joblib'

print('ncpus = '+str(ncpus))

ncpus = 15


### Connect to the datacube

In [5]:
dc = datacube.Datacube(app='prediction')

## Open the model



In [6]:
model = load(model_path)

## Open 'tiles' shapefile

In [7]:
#read shapefile
gdf = gpd.read_file('../crop_mask/data/tiles.shp')

#open shapefile
aez=gpd.read_file('../crop_mask/data/AEZs/Southern.shp')

# clip points to region
gdf = gpd.overlay(gdf, aez, how='intersection')

# add an ID column
gdf['id']=range(0, len(gdf))


In [8]:
#print gdf
list_of_tiles = [3,6,9,11,17,26]

map_shapefile(gdf.iloc[list_of_tiles], 'id', hover_col='id')

Label(value='')

Map(center=[-33.51664688303821, 20.582205531194205], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Make a prediction

Extract data from the datacube exactly matching the feature layers we created during the extraction of training data in script `1_Extract_training_data.ipynb`

In [12]:
def xr_terrain(da, attribute=None):
    """
    Using the richdem package, calculates terrain attributes
    on a DEM stored in memory as an xarray.DataArray 
    
    Params
    -------
    da : xr.DataArray
    attribute : str
        One of the terrain attributes that richdem.TerrainAttribute()
        has implemented. e.g. 'slope_riserun', 'slope_percentage', 'aspect'.
        See all option here:  
        https://richdem.readthedocs.io/en/latest/python_api.html#richdem.TerrainAttribute
        
    """
    #remove time if its there
    da = da.squeeze()
    #convert to richdem array
    rda = rd.rdarray(da.data, no_data=da.attrs['nodata'])
    #add projection and geotransform
    rda.projection=pyproj.crs.CRS(da.attrs['crs']).to_wkt()
    rda.geotransform = da.geobox.affine.to_gdal()
    #calulate attribute
    attrs = rd.TerrainAttribute(rda, attrib=attribute)

    #return as xarray DataArray
    return xr.DataArray(attrs,
                        attrs=da.attrs,
                        coords={'x':da.x, 'y':da.y},
                        dims=['y', 'x'])

def two_epochs_gm_mads(ds):
    dc = datacube.Datacube(app='training')
    
    ds1 = ds.sel(time=slice('2019-01', '2019-06'))
    ds2 = ds.sel(time=slice('2019-07', '2019-12')) 
    
    def fun(ds, era):
        print('  geomedian')
        gm = xr_geomedian(ds).compute()
        gm = calculate_indices(gm,
                               index=['NDVI', 'LAI'],
                               drop=False,
                               collection='s2')
        gm = gm.rename({
                 'blue':'blue_'+era,
                 'green':'green_'+era,
                 'red':'red_'+era,
                 'nir':'nir_'+era,
                 'swir_1':'swir_1_'+era,
                 'swir_2':'swir_2_'+era,
                 'NDVI':'NDVI_'+era,
                 'LAI':'LAI_'+era
                  })
        
        print('  TMADs')
        stats = TernaryMAD(num_threads=ncpus)
        mad = stats.compute(data=ds)
        mad.coords['x'] = ds.x
        mad.coords['y'] = ds.y
        mad = mad.rename({
            'sdev':'sdev_'+era,
            'edev':'edev_'+era,
            'bcdev':'bcdev_'+era
        })
        
        return gm,mad
    
    epoch1_gm, epoch1_mad = fun(ds1, era='S1')
    epoch2_gm, epoch2_mad = fun(ds2, era='S2')
    
    print('   Slope')
    slope = dc.load(product='srtm', like=ds.geobox).squeeze()
    slope = slope.elevation
    slope = xr_terrain(slope, 'slope_riserun')
    slope = slope.to_dataset(name='slope')
    print('   merge')
    result = xr.merge([epoch1_gm,
                       epoch1_mad,
                       epoch2_gm,
                       epoch2_mad,
                       slope], compat='override')

    return result.squeeze()

In [11]:
#set up our inputs to collect_training_data
products =  ['s2_l2a']
time = ('2019-01','2019-12')

# Set up the inputs for the ODC query
measurements =  ['red', 'green', 'nir', 'blue', 'swir_1', 'swir_2']
resolution = (-20,20)
output_crs='epsg:6933'
dask_chunks={'x':1000,'y':1000,'time':-1}

In [13]:
tiles_classified = []

for index, row in gdf.iloc[list_of_tiles].iterrows():

    print("Working on tile: "+str(gdf['id'][index]))
    
    # generate a datacube query object
    query = {
        'time': time,
        'measurements': measurements,
        'resolution': resolution,
        'output_crs': output_crs,
        'group_by' : 'solar_day',
    }
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom})

    ds = load_ard(dc=dc,
                  products=products,
                  dask_chunks=dask_chunks,
                  **query)

    data = two_epochs_gm_mads(ds)
    
    #predict using the imported model
    print('predicting...')
    predicted = predict_xr(model, data.squeeze(), progress=True)
    tiles_classified.append(predicted)    
    write_cog(predicted, 'results/classifications/Southern_'+ str(row['id'])+'_prediction.tif')
    
#     predicted_proba = predict_proba_xr(model, data.squeeze(), progress=True)
#     write_cog(predicted_proba, 'results/classifications/Southern_'+ row['id']+'_prediction_proba.tif')
    

Working on tile: 3
Using pixel quality parameters for Sentinel 2
Finding datasets
    s2_l2a
Applying pixel quality/cloud mask
Returning 145 time steps as a dask array
  geomedian
  TMADs


NotImplementedError: The da.median function only works along an axis.  The full algorithm is difficult to do in parallel